## Imports and Setup

In [1]:
#@Imports
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload

import collections
import concurrent.futures
import datetime
import json
import termcolor
import re

import numpy as np

from IPython import display

from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia import components as generic_components
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.typing import agent as simulacrum_agent
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from concordia.utils import plotting

from examples.custom_components.setup import model_setup, measurement_setup

In [2]:
#@Model Setup
model, embedder = model_setup('llama3', local_models=True, streaming=False)
measurements, clock = measurement_setup(
  SETUP_TIME=datetime.datetime(hour=21, year=2024, month=10, day=1), 
  time_step = datetime.timedelta(minutes=5))

#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

## Scenario Details

In [3]:
#@Shared Memories

# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
scenario_premise = [
  (
    'It is 9pm on a weeknight. ' +
    'Alice returns late from work and finds her friend Dorothy at her doorstep. '
  ),
]
shared_context = model.sample_text(
  'Summarize the following passage in a concise and insightful fashion:\n'
  + '\n'.join(scenario_premise)
  + '\n'
  + 'Summary:'
)

#@title Creating character backgrounds, goals and traits. Modify to explore how it influences the outcomes

player_configs = [
  
  # Dorothy
  formative_memories.AgentConfig(
    name='Dorothy',
    gender='female',
    goal=(
      'Dorothy has to move out of her house tonight and needs someone to help her move. ' +
      "Dorothy had planned on Bob helping her move, but Bob didn't show up. " 
    ),
    context=shared_context,
    traits='responsibility: medium; aggression: medium',
    extras={
      'memory_path': f"./data/Dorothy.pkl"
    }
  ),

  # Alice
  formative_memories.AgentConfig(
    name='Alice',
    gender='female',
    goal='Alice is exhausted after work, and she needs to help her grandmother all day the next day.',
    context=shared_context,
    traits='responsibility: medium; aggression: medium',
    extras={
      'memory_path': f"./data/Alice.pkl"
    }
  )
]

#@Memory factories
blank_memory_factory = blank_memories.MemoryFactory(
  model=model,
  embedder=embedder,
  importance=importance_model.importance,
  clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
  model=model,
  shared_memories=scenario_premise,
  blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

## Agent Structure

In [4]:
#@Define agent structure
%autoreload 2
from examples.custom_components import tpb_llama3 as tpb
from examples.custom_components import utils

def build_agent(player_config: formative_memories.AgentConfig) -> basic_agent.BasicAgent:

  if hasattr(player_config, 'extras'):
    memory_path = player_config.extras['memory_path'] if 'memory_path' in player_config.extras.keys() else None
  else:
    memory_path = None

  if memory_path is None:
    mem = formative_memory_factory.make_memories(player_config)
  else:
    mem = utils.load_memories(blank_memory_factory.make_blank_memory(), memory_path)

  memory = tpb.components.BasicEpisodicMemory(
    model=model,
    memory=mem,
    player_config=player_config,
    clock_now=clock.now,
    timeframe=clock.get_step_size(),
    verbose=True
  )

  behav = tpb.Behaviour(
    "behaviour",
    model=model,
    memory=mem,
    player_config=player_config,
    num_behavs=5,
    verbose=True
  )

  attitude = tpb.Attitude(
    "attitude",
    model=model,
    memory=mem,
    player_config=player_config,
    components=[behav],
    verbose=True
  )

  ############################
  # SUBJECTIVE NORM PIPELINE #

  people = tpb.People(
    "people",
    model=model,
    memory=mem,
    player_config=player_config,
    components=[behav],
    verbose=True
  )

  motiv = tpb.Motivation(
    "motivation",
    model=model,
    memory=mem,
    player_config=player_config,
    components=[people],
    verbose=True
  )

  norm = tpb.SubjectiveNorm(
    "norm",
    model=model,
    memory=mem,
    player_config=player_config,
    components=[motiv],
    verbose=True
  )
  #                          #
  ############################

  thin_goal = tpb.ThinGoal(
    "thin_goal",
    model=model,
    memory=mem,
    player_config=player_config,
    verbose=True
  )

  tpb_goal = tpb.TPB(
    "tpb",
    model=model,
    memory=mem,
    player_config=player_config,
    components=[attitude,norm,thin_goal],
    verbose=True
  )

  situation = components.situation_perception.SituationPerception(
    name='situation',
    model=model,
    memory=mem,
    agent_name = player_config.name,
    clock_now=clock.now,
    verbose=True
  )

  plan = tpb.plan.SimPlan(
    model=model,
    memory=mem,
    agent_name=player_config.name,
    clock_now=clock.now,
    components=[situation],
    goal=tpb_goal,
    time_adverb='5-minute',
    timescale='the next hour',
    verbose=True
  )

  full_plan = tpb.SequentialTPBModel(
    name='full_plan',
    components=[
      memory,
      behav,
      attitude,
      people,
      motiv,
      norm,
      tpb_goal,
      situation,
      plan
    ]
  )

  agent = basic_agent.BasicAgent(
    agent_name=player_config.name,
    model=model,
    memory=mem,
    clock=clock,
    components=[full_plan],
    update_interval=datetime.timedelta(minutes=0),
    verbose=True
  )
  
  return agent

SyntaxError: unterminated string literal (detected at line 148) (tpb_llama3.py, line 148)

In [ ]:
#@Build agents
NUM_PLAYERS = 2
agents: list[basic_agent.BasicAgent] = []
with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
  for agent in pool.map(build_agent,
            player_configs[:NUM_PLAYERS]):
    agents.append(agent)

TypeError: BasicEpisodicMemory.__init__() missing 1 required positional argument: 'player_config'

## GM Structure

In [ ]:
#@Define GM structure
game_master_memory = associative_memory.AssociativeMemory(
   sentence_embedder=embedder,
   importance=importance_model_gm.importance,
   clock=clock.now)

agent_list = [agent.name for agent in agents]

scenario_knowledge = generic_components.constant.ConstantComponent(
    state=shared_context,
    name='Background')

player_status = gm_components.player_status.PlayerStatus(
    clock_now=clock.now,
    model=model,
    memory=game_master_memory,
    player_names=agent_list,
    verbose=True)


convo_externality = gm_components.conversation.Conversation(
    players=agents,
    model=model,
    memory=game_master_memory,
    clock=clock,
    burner_memory_factory=blank_memory_factory,
    components=[player_status],
    cap_nonplayer_characters=3,
    shared_context=shared_context,
    verbose=True,
)

direct_effect_externality = gm_components.direct_effect.DirectEffect(
    players=agents,
    model=model,
    memory=game_master_memory,
    clock_now=clock.now,
    verbose=False,
    components=[player_status]
)

relevant_events = gm_components.relevant_events.RelevantEvents(
    clock.now, model, game_master_memory)
time_display = gm_components.time_display.TimeDisplay(clock)

In [ ]:
# @title Create the game master object
env = game_master.GameMaster(
    model=model,
    memory=game_master_memory,
    clock=clock,
    players=agents,
    components=[
        scenario_knowledge,
        player_status,
        convo_externality,
        direct_effect_externality,
        relevant_events,
        time_display,
    ],
    randomise_initiative=False,
    player_observes_event=False,
    players_act_simultaneously=False,
    verbose=True,
)

## Scenario

In [ ]:
game_master_memory.add(shared_context)
for agent in agents:
  agent.observe(shared_context)
# @title Expect about 2-3 minutes per step.
episode_length = 1  # @param {type: 'integer'}
for _ in range(episode_length):
  CALL_TO_ACTION = (
    "Given the above, what will {agent_name} do for the "
    "next {timedelta} to achieve their current plan?"
  )
  env.step(action_spec = simulacrum_agent.ActionSpec(call_to_action=CALL_TO_ACTION,output_type="FREE"))

Updating game master components
Events:
[01 Oct 2024 21:00:00] A unexpected visit: At 9pm on a typical weeknight, Alice returns home from work to find her friend Dorothy waiting at the doorstep, an unusual occurrence that sets the stage for what's to come.

The current time is: [01 Oct 2024 21:00:00]

Question: Given the above events and their time, what is the latest location of Dorothy and what are they doing?
Answer: Dorothy is Based on the event "A unexpected visit", we know that Alice returns home from work to find Dorothy waiting at the doorstep. Since it's 9pm (21:00), we can infer that Dorothy has arrived at Alice's location, which is likely her home or a nearby spot where they planned to meet.

So, the latest location of Dorothy is likely Alice's home or a nearby area, and they are probably having an unexpected visit, possibly discussing something important or catching up.

Events:
[01 Oct 2024 21:00:00] A unexpected visit: At 9pm on a typical weeknight, Alice returns home fro

KeyboardInterrupt: 

Output: Alice -- "Hey Dorothy, sorry about this but I really need a minute to check my phone real quick. Can we catch up in just a sec?"

Alice context of action:
Alice's full_plan:
Here is a rewritten version of Alice's current goal in terms of a single concrete action that can be taken or started within the next five minutes:

"Quickly check and update my phone for any important messages or notifications, then excuse myself to Dorothy and politely tell her I need a few minutes to catch up before we chat further."

Question: Given the above, what is Alice likely to say next? Respond in the format `Alice -- "..."` For example, Cristina -- "Hello! Mighty fine weather today, right?", Ichabod -- "I wonder if the alfalfa is ready to harvest", or Townsfolk -- "Good morning".

Answer: Alice -- "Hey Dorothy, sorry about this but I really need a minute to check my phone real quick. Can we catch up in just a sec?"

Initializing player update.
Updating game master components.
Printing game maste

In [ ]:
import ipywidgets as widgets

agent_tabs = ["Game Master", "Alice", "Dorothy"]
children = [widgets.Text(description=agent_tab) for agent_tab in agent_tabs]
tab = widgets.Tab()
tab.children = children
tab.titles = [agent_tab for agent_tab in agent_tabs]
tab


### Things to add

- Summarize deliberation on the potential actions, including thoughts on the possible positive and negative consequences (counterfactuals should remain throughout the whole)
- 